## 🧠 LBP (Local Binary Pattern)이란?

### 📌 개념
- LBP는 픽셀 주변의 밝기 변화를 이진 코드로 표현해서 텍스처(무늬) 특징을 추출하는 방법.
- 중심 픽셀보다 밝은 이웃은 1, 어두운 이웃은 0으로 표시 → 8비트 이진수 → 0~255 값으로 변환.

### ✅ 얼굴 인식에 적합한 이유
- 조명 변화에 강하고, 얼굴의 텍스처 특징을 잘 잡아냄.
- Haar보다 가볍고 빠르며, 실시간 얼굴 검출에 적합.

---

## 🧩 LBP 기반 얼굴 인식 전체 흐름

### 🎯 목표: 얼굴을 검출하고, 누구인지까지 식별하기

| 단계 | 설명 |
|------|------|
| 1단계 | `lbpcascade_frontalface.xml`을 이용해서 얼굴 검출 |
| 2단계 | 검출된 얼굴 영역(ROI)을 잘라냄 |
| 3단계 | 얼굴 이미지를 흑백 변환하고 크기 정규화 (예: 100x100) |
| 4단계 | LBP 특징을 추출하고, SVM이나 LBPH로 학습 |
| 5단계 | 새로운 얼굴 이미지가 들어오면 같은 방식으로 특징 추출 후, 분류기로 예측 |

---

## 🛠️ 사용 기술 구성

| 항목 | 사용 기술 |
|------|-----------|
| 얼굴 검출 | OpenCV LBP Cascade (`lbpcascade_frontalface.xml`) |
| 특징 추출 | Local Binary Pattern (LBP) |
| 인식 모델 | LBPHFaceRecognizer / SVM / KNN 등 |
| 입력 방식 | 이미지, 웹캠, CCTV 등 |
| 출력 방식 | 얼굴 위치 + 이름(혹은 ID) 출력

---

## 🧪 예시 활용

- 📸 출입 통제: 얼굴 인식으로 문 열림
- 🎓 출석 체크: 자동으로 학생 이름 인식
- 🔐 개인화 시스템: "홍길동님, 환영합니다" 같은 메시지 출력
- 📷 CCTV 모니터링: 등록된 사람만 자동 식별

---

## ⚠️ 주의할 점

- 정면 얼굴일 때 성능이 가장 좋음
- 얼굴 이미지의 밝기/크기/해상도 정규화가 중요함
- 인식 성능은 학습 데이터 양과 품질에 크게 영향을 받음
- LBP는 전통 방식이라, 고정된 조명/환경에서 가장 잘 작동함

---

## 🚀 대안 기술

- 🔍 더 높은 성능을 원한다면, 딥러닝 기반 기술도 고려해볼 수 있음
  - 예: FaceNet, ArcFace, Dlib(128D), DeepFace 등
  - 다양한 각도,


In [3]:
import cv2
import numpy as np
import os
from PIL import Image

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create() # LBPH를 사용할 새 변수 생성

Face_ID = -1
prev_person_name = ""
y_ID = []
x_train = []

Face_Images = os.path.join(os.getcwd(), 'Face_Images') # 이미지 폴더 지정
print(Face_Images)

for root, dirs, files in os.walk(Face_Images):
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'):
            path = os.path.join(root, file)
            person_name = os.path.basename(root)
            print(path, person_name)

        if prev_person_name != person_name:
            Face_ID += 1
            prev_person_name = person_name

        img = cv2.imread(path)
        gray_iamge = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_iamge, scaleFactor=1.3, minNeighbors=5)

        print(Face_ID, faces)

        for (x,y,w,h) in faces:
            roi = gray_iamge[y:y+h, x:x+w]
            x_train.append(roi)
            y_ID.append(Face_ID)

            recognizer.train(x_train, np.array(y_ID))
            recognizer.save('face-trainner.yml')

c:\Users\main\Desktop\vision_project\ch02\Face_Images
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\1.jpg Cha
0 [[147 140 278 278]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\2.jpg Cha
0 [[213 183 352 352]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\3.jpg Cha
0 [[122 201 360 360]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\4.jpg Cha
0 [[505 100 226 226]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\5.jpg Cha
0 [[415 158 412 412]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\cha1.jpg Cha
0 [[285 205 452 452]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\cha2.jpg Cha
0 ()
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\cha3.jpg Cha
0 ()
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\cha4.jpg Cha
0 ()
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Cha\cha5.jpg Cha
0 ()
c:\Users\main\Desktop\vision_project\ch02\Face_Images\Ma\ma1.jpg Ma
1 [[ 71 105 472 472]]
c:\Users\

## 일부 사진을 검출하지 못하는 이유
* print(Face_ID, faces) 출력문에서 faces 가 없다는건 얼굴 이미지가 인식하지못했다는 의미
* 검출하지못한 이미지를 살펴보면 얼굴을 가리고있음 (손으로 얼굴을 가림)
* 얼굴이 정면이 아닌 측면임 (haar cascade 의 단점)

In [6]:
import cv2
import numpy as np
import os
from PIL import Image

labels = ['Cha', 'Ma']

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create() # LBPH를 사용할 새 변수 생성
recognizer.read('face-trainner.yml') # 학습한 값 읽어오기

image_list = []

test_images = os.path.join(os.getcwd(), 'test_list')

for root, dirs, files in os.walk(test_images):
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'):
            image_path = os.path.join(test_images, file)
            print(image_path)
            image_list.append(cv2.imread(image_path))

for img in image_list:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x,y,w,h)in faces:
        roi_gray = gray[y:y+h, x:x+w]

        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)

        if conf >= 50:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(img, name, (x,y), font, 1, (0,0,255), 2)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)

    cv2.imshow('Preview', img)
    if cv2.waitKey(0) >= 0:
        continue

cv2.destroyAllWindows()

        

c:\Users\main\Desktop\vision_project\ch02\test_list\cha_test1.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list\cha_test2.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list\ma_test1.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list\ma_test2.jpg
Cha 40.54975582829157
Ma 42.30553092812899
Cha 53.44813116389822


## Test_image 도 정면이 아니면 잘 예측하지못함

# 다른 데이터로 복습

In [9]:
import cv2
import numpy as np
import os
from PIL import Image

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create() # LBPH를 사용할 새 변수 생성

Face_ID = -1
prev_person_name = ""
y_ID = []
x_train = []

Face_Images = os.path.join(os.getcwd(), 'Face_Images2') # 이미지 폴더 지정
print(Face_Images)

for root, dirs, files in os.walk(Face_Images):
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'):
            path = os.path.join(root, file)
            person_name = os.path.basename(root)
            print(path, person_name)

        if prev_person_name != person_name:
            Face_ID += 1
            prev_person_name = person_name

        img = cv2.imread(path)
        gray_iamge = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_iamge, scaleFactor=1.3, minNeighbors=5)

        print(Face_ID, faces)

        for (x,y,w,h) in faces:
            roi = gray_iamge[y:y+h, x:x+w]
            x_train.append(roi)
            y_ID.append(Face_ID)

            recognizer.train(x_train, np.array(y_ID))
            recognizer.save('face-trainner.yml')

c:\Users\main\Desktop\vision_project\ch02\Face_Images2
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Ka\1.jpg Ka
0 [[147 129 356 356]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Ka\2.jpg Ka
0 [[104 133 441 441]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Ka\3.jpg Ka
0 [[272 324 480 480]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Ka\4.jpg Ka
0 [[53 57 81 81]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Ka\5.jpg Ka
0 [[109 104 239 239]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Win\1.jpg Win
1 [[430 257 467 467]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Win\2.jpg Win
1 [[89 28 95 95]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Win\3.jpg Win
1 [[224 166 352 352]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Win\4.jpg Win
1 [[87 43 65 65]]
c:\Users\main\Desktop\vision_project\ch02\Face_Images2\Win\5.jpg Win
1 [[204  64 132 132]]


In [22]:
import cv2
import numpy as np
import os
from PIL import Image

labels = ['Karina', 'Winter']

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create() # LBPH를 사용할 새 변수 생성
recognizer.read('face-trainner.yml') # 학습한 값 읽어오기

image_list = []

test_images = os.path.join(os.getcwd(), 'test_list2')

for root, dirs, files in os.walk(test_images):
    for file in files:
        if file.endswith('jpeg') or file.endswith('jpg') or file.endswith('png'):
            image_path = os.path.join(test_images, file)
            print(image_path)
            image_list.append(cv2.imread(image_path))

for img in image_list:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x,y,w,h)in faces:
        roi_gray = gray[y:y+h, x:x+w]

        id_, conf = recognizer.predict(roi_gray)
        print(labels[id_], conf)

        if conf >= 40:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            cv2.putText(img, name, (x,y), font, 1, (0,0,255), 2)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)

    cv2.imshow('Preview', img)
    if cv2.waitKey(0) >= 0:
        continue

cv2.destroyAllWindows()

        

c:\Users\main\Desktop\vision_project\ch02\test_list2\ka1.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list2\ka2.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list2\win1.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list2\win2.jpg
c:\Users\main\Desktop\vision_project\ch02\test_list2\win3.jpg
Karina 89.16544952350635
Winter 78.79645250793659
Winter 88.40998433741618
Karina 63.73462932569462
